# Part 1- Gather Data and Images

## Using TMDB API Gathers Movies and their Poster Paths

In [2]:
import requests
import json
import io
from time import time, sleep
import numpy as np
import math

# Get ID's of Popular movies

In [108]:
import http.client
popId=[]
for i in range(0,15):
    startTime = time()
    pages=np.arange(39*i+1,39*(i+1)+1)
    for p in pages:
        conn = http.client.HTTPSConnection("api.themoviedb.org")
        payload = "{}"
        conn.request("GET", "/3/movie/popular?page="+str(p)+"&language=en-US&api_key=c31abda47a98cd26c2c8ad333a3db035", payload)
        res = conn.getresponse()
        data = res.read()
        data=data.decode("utf-8")
        hold=json.loads(data)
        for j in hold["results"]:
            popId.append(j["id"])
        conn.close()
    endTime=time()-startTime
    print (i,endTime)
    if endTime<10.0:
        sleep(10.0-endTime)
    else:
        continue


0 4.824997901916504
1 5.2702107429504395
2 4.761780023574829
3 5.00071120262146
4 4.800471067428589
5 4.700629949569702
6 4.543933868408203
7 4.4542460441589355
8 4.773443937301636
9 4.626511812210083
10 4.528390884399414
11 5.102146148681641
12 4.780484914779663
13 4.574825048446655
14 4.6884942054748535


In [109]:
print (len(popId))
End=math.ceil(len(popId)/40)
print(End)

11700
293


In [113]:
target = io.open("tmdbMovies.csv", 'w',encoding="utf8")
line="title;imdb ID;TMDB ID;poster path;genre count;Genres" ##IF APPENDING DONT INCLUDE 
target.write(line+"\n")
payload = "{}"
count=0
for i in range(0,End+1):
    startTime = time()
    movie_ids=popId[40*i:40*(i+1)]
    for movie_id in movie_ids:
        #url = "https://api.themoviedb.org/3/movie/"
        url = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"?api_key=c31abda47a98cd26c2c8ad333a3db035"
        response = requests.request("GET", url, data=payload)
        #print (response.text)
        data=json.loads(str(response.text))
        if("status_code" in data):
            pass
        elif(not"adult"=="true"):
            count=count+1
            line=""
            line=line+str(data["original_title"])+";"
            line=line+str(data["imdb_id"])+";"
            line=line+str(data["id"])+";"
            line=line+str(data["poster_path"])+";"
            genres_string=""
            countGenre=0
            for g in data['genres']:
                countGenre=countGenre+1;
                genres_string=genres_string+"~"+str(g['name'])
            line=line+str(countGenre)+";"
            line=line+genres_string
            #print(line)
            target.write(line+"\n")
    target.flush() 
    endTime=time()-startTime
    print (i,endTime)
    if endTime<10.0:
        sleep(10.0-endTime)
    else:
        continue
target.close()
print("Actually got ",count," movies")  

0 4.056813955307007
1 4.700993061065674
2 3.890277147293091
3 4.343744993209839
4 3.7965087890625
5 4.249590873718262
6 4.07924485206604
7 3.8198180198669434
8 3.88393497467041
9 4.05160117149353
10 4.0117270946502686
11 3.9491281509399414
12 4.8164098262786865
13 6.455536842346191
14 6.777601003646851
15 4.546689987182617
16 3.7094669342041016
17 4.625098943710327
18 4.076001167297363
19 4.0977349281311035
20 4.682670831680298
21 4.340892791748047
22 4.337123155593872
23 4.72179102897644
24 3.91565203666687
25 4.2301530838012695
26 4.19743800163269
27 3.882469892501831
28 4.043738126754761
29 4.068946838378906
30 6.8503663539886475
31 4.002461910247803
32 4.071398973464966
33 4.6642467975616455
34 4.3242151737213135
35 4.223398923873901
36 4.164045095443726
37 4.4240899085998535
38 4.657742977142334
39 4.175616025924683
40 4.09067177772522
41 3.9593112468719482
42 4.2603440284729
43 4.251363754272461
44 4.017435073852539
45 4.2280519008636475
46 4.120904207229614
47 4.767812013626099


# Opens csv file in Pandas

In [3]:
import pandas as pd
movies = pd.read_csv("tmdbMovies.csv", delimiter=';', error_bad_lines=False, skip_blank_lines =True,encoding = "ISO-8859-1")
movies = movies[np.isfinite(movies['genre count'])]
slc=["TMDB ID","genre count"]
movies=movies.astype({c: int for c in slc})
movies.head(20)

b'Skipping line 8821: expected 6 fields, saw 7\n'


,title,imdb ID,TMDB ID,poster path,genre count,Genres
0,Fifty Shades Freed,tt4477536,337167,/jjPJ4s3DWZZvI4vw8Xfi4Vqa1Q8.jpg,2,~Drama~Romance
1,Zootopia,tt2948356,269149,/sM33SANp9z6rXW8Itn7NnG1GOEs.jpg,4,~Animation~Adventure~Family~Comedy
2,Avengers: Infinity War,tt4154756,299536,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,4,~Adventure~Science Fiction~Fantasy~Action
3,Coco,tt2380307,354912,/eKi8dIrr8voobbaGzDpe8w0PVbC.jpg,4,~Adventure~Comedy~Family~Animation
4,Rampage,tt2231461,427641,/30oXQKwibh0uANGMs0Sytw3uN22.jpg,3,~Action~Adventure~Science Fiction
5,Black Panther,tt1825683,284054,/uxzzxijgPIY7slzFvMotPv8wjKA.jpg,4,~Action~Adventure~Fantasy~Science Fiction
6,Beauty and the Beast,tt2771200,321612,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,3,~Family~Fantasy~Romance
7,Star Wars: The Last Jedi,tt2527336,181808,/kOVEVeg59E0wsnXmF9nrh6OmWII.jpg,3,~Fantasy~Adventure~Science Fiction
8,A Quiet Place,tt6644200,447332,/nAU74GmpUk7t5iklEp3bufwDq4n.jpg,4,~Drama~Horror~Thriller~Science Fiction
9,The Maze Runner,tt1790864,198663,/coss7RgL0NH6g4fC2s5atvf3dFO.jpg,4,~Action~Mystery~Science Fiction~Thriller


# Creating Final Data Set and Processing Genres

In [4]:

genresList=[]
url = "https://api.themoviedb.org/3/genre/movie/list?language=en-US&api_key=c31abda47a98cd26c2c8ad333a3db035"

payload = "{}"
response = requests.request("GET", url, data=payload)
data=json.loads(str(response.text))
for g in data['genres']:
    genresList.append(str(g['name']))


Finaldata=pd.DataFrame()
for i in range(len(movies.index)): #SIZE OF DATAFRAME
    temp=movies.iloc[i]
    #print(temp["TMDB ID"])
    if(not temp["genre count"]==0): 
        if (not temp["poster path"]=="None"):
            hold=temp["Genres"].split("~")
            hold=hold[1:]
            hold = [item.strip() for item in hold]
            #print(hold)
            new_dict = {'TMDB ID':temp["TMDB ID"],'poster path':temp["poster path"],'Action': 0, 'Adventure': 0, 'Animation': 0,'Comedy': 0, 'Crime': 0, 
                'Documentary': 0,'Drama': 0, 'Family': 0, 'Fantasy': 0,'History': 0, 'Horror': 0, 'Music': 0,
               'Mystery': 0, 'Romance': 0, 'Science Fiction': 0,'TV Movie': 0, 'Thriller': 0, 'War': 0,'Western':0}
            for movieG in hold:
                if movieG in genresList:
                    new_dict[movieG]=1
            
            hol2=pd.DataFrame([new_dict],columns=list(new_dict.keys()))
            Finaldata=Finaldata.append(hol2)
        

In [5]:
Finaldata=Finaldata.reset_index()
Finaldata=Finaldata.drop(columns=['index'])
Finaldata.head(10)

,TMDB ID,poster path,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,337167,/jjPJ4s3DWZZvI4vw8Xfi4Vqa1Q8.jpg,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,269149,/sM33SANp9z6rXW8Itn7NnG1GOEs.jpg,0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,299536,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,354912,/eKi8dIrr8voobbaGzDpe8w0PVbC.jpg,0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,427641,/30oXQKwibh0uANGMs0Sytw3uN22.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,284054,/uxzzxijgPIY7slzFvMotPv8wjKA.jpg,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6,321612,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
7,181808,/kOVEVeg59E0wsnXmF9nrh6OmWII.jpg,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
8,447332,/nAU74GmpUk7t5iklEp3bufwDq4n.jpg,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,1,0,0
9,198663,/coss7RgL0NH6g4fC2s5atvf3dFO.jpg,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In [8]:
print (Finaldata.shape)

(11062, 21)


# Create a csv file of Finaldata

In [7]:
Finaldata.to_csv("Finaldata.csv", encoding='utf-8', index=False,sep=";")


# Downloading Images via Path Using Final dataset

In [9]:

import urllib.request
count =0;
imagesDownload=0
for i in Finaldata["poster path"]:
    if not i=="None":
        url="https://image.tmdb.org/t/p/w500"+i
        #print (url)
        saveName="Pictures/"+str(Finaldata["TMDB ID"][count]) +".jpg"
        #print(saveName)
        urllib.request.urlretrieve(url,saveName)
        imagesDownload=imagesDownload+1;
    count=count+1;
print("The images downloaded",imagesDownload)    

The images downloaded 11062
